In [1]:
# Import dependencies
import pandas as pd
import math
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read data
plays_df = pd.read_csv('https://storage.googleapis.com/big-data-bowl/plays.csv')
players_df = pd.read_csv('https://storage.googleapis.com/big-data-bowl/players.csv')
pffscouting_df = pd.read_csv('https://storage.googleapis.com/big-data-bowl/pffScoutingData.csv')
games_df = pd.read_csv('https://storage.googleapis.com/big-data-bowl/games.csv')

print("Plays: "  + str(plays_df.shape))
print("Players: "  + str(players_df.shape))
print("Scouting: "  + str(pffscouting_df.shape))
print("Games: "  + str(games_df.shape))

Plays: (8557, 32)
Players: (1679, 7)
Scouting: (188254, 15)
Games: (122, 7)


In [3]:
combined_df = pd.merge(pffscouting_df, plays_df,  how='left', left_on=['gameId','playId'], right_on=['gameId','playId']) 
combined_df = pd.merge(combined_df, players_df, how='left', left_on=['nflId'], right_on=['nflId']) 
combined_df = pd.merge(combined_df, games_df, how='left', left_on=['gameId'], right_on=['gameId']) 


combined_df['birthDate'].replace('NA','', inplace=True)
combined_df['birthDate'] = pd.to_datetime(combined_df['birthDate'])
combined_df['age'] = combined_df['birthDate'].map(lambda x: dt.date.today().year-x.year)
combined_df['age'] = combined_df['age'].fillna(0)
# combined_df = combined_df.drop(columns='birthDate')
combined_df['heightCm'] = combined_df['height'].map(lambda x: int(x.split('-')[0])*30.48+int(x.split('-')[1])*2.54)
# combined_df = combined_df.drop(columns='height')

combined_df['posTeamIsHomeTeam'] = 0
combined_df.loc[combined_df['possessionTeam'] == combined_df['homeTeamAbbr'], 'posTeamIsHomeTeam'] = 1

combined_df['playerCommittedFoul'] = 0
combined_df.loc[combined_df['foulNFLId1'] == combined_df['nflId'], 'playerCommittedFoul'] = 1 
combined_df.loc[combined_df['foulNFLId2'] == combined_df['nflId'], 'playerCommittedFoul'] = 1 
combined_df.loc[combined_df['foulNFLId3'] == combined_df['nflId'], 'playerCommittedFoul'] = 1 

combined_df['gameClockInside2Minutes'] = 0
combined_df['gameClockInside2Minutes'] = combined_df['gameClock'].map(lambda x: int(x.split(':')[0]) < 2) 

combined_df

print('dataframe size: {}'.format(combined_df.shape))
combined_df

dataframe size: (188254, 62)


,gameId,playId,nflId,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,...,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,age,heightCm,posTeamIsHomeTeam,playerCommittedFoul,gameClockInside2Minutes
0,2021090900,97,25511,Pass,QB,NaN,NaN,NaN,NaN,NaN,...,1,09/09/2021,20:20:00,TB,DAL,45.0,193.04,1,0,False
1,2021090900,97,35481,Pass Route,TE-L,NaN,NaN,NaN,NaN,NaN,...,1,09/09/2021,20:20:00,TB,DAL,33.0,198.12,1,0,False
2,2021090900,97,35634,Pass Route,LWR,NaN,NaN,NaN,NaN,NaN,...,1,09/09/2021,20:20:00,TB,DAL,34.0,177.80,1,0,False
3,2021090900,97,39985,Pass Route,HB-R,NaN,NaN,NaN,NaN,NaN,...,1,09/09/2021,20:20:00,TB,DAL,31.0,175.26,1,0,False
4,2021090900,97,40151,Pass Block,C,NaN,NaN,NaN,0.0,0.0,...,1,09/09/2021,20:20:00,TB,DAL,31.0,193.04,1,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188249,2021110100,4433,52507,Pass Block,LT,NaN,NaN,NaN,0.0,0.0,...,8,11/01/2021,20:15:00,KC,NYG,25.0,200.66,0,0,True
188250,2021110100,4433,52546,Coverage,SCBoR,0.0,0.0,0.0,NaN,NaN,...,8,11/01/2021,20:15:00,KC,NYG,25.0,185.42,0,0,True
188251,2021110100,4433,52573,Pass Route,SLoWR,NaN,NaN,NaN,NaN,NaN,...,8,11/01/2021,20:15:00,KC,NYG,25.0,198.12,0,0,True
188252,2021110100,4433,52585,Pass Rush,LEO,0.0,0.0,0.0,NaN,NaN,...,8,11/01/2021,20:15:00,KC,NYG,0.0,187.96,0,0,True


In [4]:
combined_df.columns

Index(['gameId', 'playId', 'nflId', 'pff_role', 'pff_positionLinedUp',
       'pff_hit', 'pff_hurry', 'pff_sack', 'pff_beatenByDefender',
       'pff_hitAllowed', 'pff_hurryAllowed', 'pff_sackAllowed',
       'pff_nflIdBlockedPlayer', 'pff_blockType', 'pff_backFieldBlock',
       'playDescription', 'quarter', 'down', 'yardsToGo', 'possessionTeam',
       'defensiveTeam', 'yardlineSide', 'yardlineNumber', 'gameClock',
       'preSnapHomeScore', 'preSnapVisitorScore', 'passResult', 'penaltyYards',
       'prePenaltyPlayResult', 'playResult', 'foulName1', 'foulNFLId1',
       'foulName2', 'foulNFLId2', 'foulName3', 'foulNFLId3',
       'absoluteYardlineNumber', 'offenseFormation', 'personnelO',
       'defendersInBox', 'personnelD', 'dropBackType', 'pff_playAction',
       'pff_passCoverage', 'pff_passCoverageType', 'height', 'weight',
       'birthDate', 'collegeName', 'officialPosition', 'displayName', 'season',
       'week', 'gameDate', 'gameTimeEastern', 'homeTeamAbbr',
       'visit

In [5]:
# Filter for offensive line positions
combined_df_oline = combined_df[combined_df['officialPosition'].isin(['G','C','T'])]
print("Offensive Line Plays: "  + str(combined_df_oline.shape))
combined_df_oline

Offensive Line Plays: (42886, 62)


,gameId,playId,nflId,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,...,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,age,heightCm,posTeamIsHomeTeam,playerCommittedFoul,gameClockInside2Minutes
4,2021090900,97,40151,Pass Block,C,NaN,NaN,NaN,0.0,0.0,...,1,09/09/2021,20:20:00,TB,DAL,31.0,193.04,1,0,False
7,2021090900,97,42377,Pass Block,LT,NaN,NaN,NaN,0.0,0.0,...,1,09/09/2021,20:20:00,TB,DAL,29.0,198.12,1,0,False
9,2021090900,97,42404,Pass Block,LG,NaN,NaN,NaN,0.0,0.0,...,1,09/09/2021,20:20:00,TB,DAL,29.0,193.04,1,0,False
16,2021090900,97,46163,Pass Block,RG,NaN,NaN,NaN,0.0,0.0,...,1,09/09/2021,20:20:00,TB,DAL,27.0,198.12,1,0,False
18,2021090900,97,52421,Pass Block,RT,NaN,NaN,NaN,0.0,0.0,...,1,09/09/2021,20:20:00,TB,DAL,23.0,195.58,1,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188232,2021110100,4433,37090,Pass Block,RT,NaN,NaN,NaN,0.0,0.0,...,8,11/01/2021,20:15:00,KC,NYG,34.0,203.20,0,0,True
188239,2021110100,4433,43695,Pass Block,LG,NaN,NaN,NaN,0.0,0.0,...,8,11/01/2021,20:15:00,KC,NYG,29.0,190.50,0,0,True
188241,2021110100,4433,46090,Pass Block,C,NaN,NaN,NaN,0.0,0.0,...,8,11/01/2021,20:15:00,KC,NYG,28.0,193.04,0,0,True
188242,2021110100,4433,46103,Pass Block,RG,NaN,NaN,NaN,0.0,0.0,...,8,11/01/2021,20:15:00,KC,NYG,27.0,187.96,0,0,True


In [6]:
# Shape the dataframe so predictions can be made
combined_df_oline_cleaned = combined_df_oline.drop(columns=['gameId', 'playId', 'pff_role', 'pff_hit','pff_hurry','pff_sack', 'playDescription', 'displayName', 'season', 'gameDate', 'gameTimeEastern',
                                                            'homeTeamAbbr', 'visitorTeamAbbr', 'officialPosition', 'foulName1', 'foulNFLId1', 'foulName2', 'foulNFLId2', 'foulName3', 'foulNFLId3',
                                                            'pff_backFieldBlock', 'height', 'birthDate', 'gameClock'])

combined_df_oline_cleaned




,nflId,pff_positionLinedUp,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,quarter,down,...,pff_passCoverage,pff_passCoverageType,weight,collegeName,week,age,heightCm,posTeamIsHomeTeam,playerCommittedFoul,gameClockInside2Minutes
4,40151,C,0.0,0.0,0.0,0.0,44955.0,SW,1,3,...,Cover-1,Man,319,Colorado State-Pueblo,1,31.0,193.04,1,0,False
7,42377,LT,0.0,0.0,1.0,0.0,42403.0,PP,1,3,...,Cover-1,Man,338,Penn State,1,29.0,198.12,1,0,False
9,42404,LG,0.0,0.0,0.0,0.0,44955.0,PP,1,3,...,Cover-1,Man,307,Hobart,1,29.0,193.04,1,0,False
16,46163,RG,0.0,0.0,1.0,0.0,53441.0,SW,1,3,...,Cover-1,Man,305,Humboldt State,1,27.0,198.12,1,0,False
18,52421,RT,0.0,0.0,1.0,0.0,53441.0,PP,1,3,...,Cover-1,Man,322,Iowa,1,23.0,195.58,1,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188232,37090,RT,0.0,0.0,0.0,0.0,52585.0,PP,4,4,...,Quarters,Zone,325,Colorado,8,34.0,203.20,0,0,True
188239,43695,LG,0.0,0.0,0.0,1.0,43338.0,SW,4,4,...,Quarters,Zone,313,Duke,8,29.0,190.50,0,0,True
188241,46090,C,0.0,0.0,0.0,0.0,NaN,NB,4,4,...,Quarters,Zone,312,Ohio State,8,28.0,193.04,0,0,True
188242,46103,RG,0.0,0.0,0.0,0.0,43326.0,PP,4,4,...,Quarters,Zone,327,Texas-El Paso,8,27.0,187.96,0,0,True


In [7]:
# Save to CSV
combined_df_oline_cleaned.to_csv('gs://big-data-bowl/play_analysis_oline.csv')